In [4]:
import serial
from func.formula_for_data import formula_for_data
from func.create_line import create_line
from func.graph import graph
import plotly.graph_objects as go
from datetime import date
from plotly.subplots import make_subplots
from numpy import zeros, vstack, append
import pandas as pd

Сначала отобразим график

In [5]:
fig = go.FigureWidget(make_subplots(rows=3, cols=1, subplot_titles=("Линейные ускорения", "Угловые скорости", "Углы")))
graph(fig)
fig

FigureWidget({
    'data': [{'legendgroup': 'group0',
              'line': {'color': 'red'},
              'm…

Сам код:

In [6]:
# Вся инфа о датчике: https://github.com/WITMOTION/BWT901CL/blob/master/BWT901CL%20Datasheet.pdf

s = serial.Serial('/dev/ttyUSB0', 115200)  # Открываем порт, для Windows: COM3

# Создадим массивы для графиков
aa = zeros(3)
ww = zeros(3)
AA = zeros(3)

df = pd.DataFrame() # Создадим таблицу, куда будем сохранять все значения
df = df.reindex(columns=['aX', 'aY', 'aZ', 'wX', 'wY', 'wZ', 'AX', 'AY', 'AZ',])

x_label_size = 50   # На графике будет отображаться 50 значений
counter = 0

while 1: # Цикл бесконечный, останавливать вручну
    if s.read().hex() != '55': # Ищем, чтобы этот байт по ASCII был равен 55, т.е. U, см. стр 13
        continue
    head = s.read().hex()
    if head == '51':
        a = formula_for_data(head, s)   # Ускорения
        end = s.read(3) # Температура и сумма (нам не надо)
        aa = vstack((aa, a))    # Присоединение новых данных по вертикали
    if head == '52':
        w = formula_for_data(head, s)   # Угловые скорости
        end = s.read(3)
        ww = vstack((ww, w))
    if head == '53':
        A = formula_for_data(head, s)   # Углы
        end = s.read(3)
        AA = vstack((AA, A))
        
        # Отрисовка графика
        with fig.batch_update():
            for i in range(3):
                fig.data[i].y = aa[:,i]
                fig.data[i+3].y = ww[:,i]
                fig.data[i+6].y = AA[:,i]
        
        if len(AA) > x_label_size:  # Ограничение по отображаемым данным
            aa = aa[-x_label_size:]
            ww = ww[-x_label_size:] 
            AA = AA[-x_label_size:]
            
        df.loc[counter,:] = a + w + A
        counter += 1 
s.close()

KeyboardInterrupt: 

In [ ]:
df.to_csv('BWT901CL_res.csv')   # Сохраним полученную таблицу